In [1]:
!pip install tinyec

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import secrets
from tinyec import registry

In [3]:
def compress(pubKey):
    return hex(pubKey.x)

In [4]:
curve = registry.get_curve('secp256r1')

In [5]:
alicePrivKey = secrets.randbelow(curve.field.n)
alicePubKey = alicePrivKey * curve.g
print("Alice public key:", compress(alicePubKey))

Alice public key: 0xb78e2e57620910f427525e5b3971833a5fe1cef062e9fcb05fdd07996627d8ab


In [6]:
bobPrivKey = secrets.randbelow(curve.field.n)
bobPubKey = bobPrivKey * curve.g
print("Bob public key:", compress(bobPubKey))

Bob public key: 0xc0d5d2ef7a7c1e15c43b1be3ee9d43ae6168f31abaac2a88c94bc829ed170ce2


# ECDH

In [7]:
aliceSharedKey = alicePrivKey * bobPubKey
print("Alice shared key:", compress(aliceSharedKey))

Alice shared key: 0x894bafaf21f9cc9fd880a8fa8e8f8e9c592d385dffd55c60a111996a5b44b4ea


In [8]:
bobSharedKey = bobPrivKey * alicePubKey
print("Bob shared key:", compress(bobSharedKey))

Bob shared key: 0x894bafaf21f9cc9fd880a8fa8e8f8e9c592d385dffd55c60a111996a5b44b4ea


In [9]:
print("Equal shared keys:", aliceSharedKey == bobSharedKey)

Equal shared keys: True


# Selected

In [10]:
aliceSessionSecret = secrets.randbelow(curve.field.n)
print("aliceSessionSecret:", aliceSessionSecret)

aliceSelfPartialShared = aliceSessionSecret * curve.g
print("aliceSelfPartialShared:", compress(aliceSelfPartialShared))

toBob = (alicePrivKey + aliceSessionSecret) * bobPubKey
print("to Bob:", compress(toBob))

aliceSessionSecret: 107823986422194310705984168630421926947990428659194648727579011669442409961706
aliceSelfPartialShared: 0x544796743a83373b999e59413254f512cde7dcb1e6dffa84695283810ac3031
to Bob: 0xbc539917da69844f45f9123982376bfed47039a0454cdf4f1fc4be9404b182e9


In [11]:
bobSessionSecret = secrets.randbelow(curve.field.n)
print("bobSessionSecret:", bobSessionSecret)

bobSelfPartialShared = bobSessionSecret * curve.g
print("bobSelfPartialShared:", compress(bobSelfPartialShared))

toAlice = (bobPrivKey + bobSessionSecret) * alicePubKey
print("to Alice:", compress(toAlice))

bobSessionSecret: 31795624434691938380819900179971381491177620673220318826992248653063820471106
bobSelfPartialShared: 0x13cb3985239bd83c71b9779edd96dd3c73944c60dda3f20d446a5bde100799e1
to Alice: 0x6374c84e8e90baa1061ac074840835609a10fc0f0e9b82dd8b7d90469ea552c1


## Computing shared

In [17]:
alicePrivKeyInv = pow(alicePrivKey, -1, curve.field.n)

alicePartialShared =  (toAlice * alicePrivKeyInv) - alicePubKey
print("alice Partial Shared:", compress(alicePartialShared))

aliceShared = aliceSelfPartialShared + alicePartialShared
print("aliceShared:", compress(aliceShared))

alice Partial Shared: 0xcbedc5691fb081b68d42d4498f7ed5f1b95db2bb84b1f3ba69af8586aab0728e
aliceShared: 0xf4ce057fbbb5b8d6ef0fa2901db7487d4c96086a95fdc4f9065a50456a387a7b


In [18]:
bobPrivKeyInv = pow(bobPrivKey, -1, curve.field.n)

bobPartialShared = (toBob * bobPrivKeyInv) - bobPubKey
print("bob Partial Shared:", compress(bobPartialShared))

bobShared = bobSelfPartialShared + bobPartialShared
print("bobShared:", compress(bobShared))

bob Partial Shared: 0xcc6583920e4209bb65573037bb70094c7619b886b6e8bb2f9f13164b92ecb914
bobShared: 0x3ef7e8e7a0db018ac663aae172dd0912651fa42f97d99e4413f47de0b0e532a5


In [20]:
print("Equal shared keys:", aliceShared.x == bobShared.x)

Equal shared keys: False
